#### Import Necessary Libraries




In [ ]:
import json
import requests
import pandas as pd
from tqdm import tqdm

from bs4 import BeautifulSoup as bs

### Scaraping Movie Information

In [ ]:
def extract_data(link):

  r = requests.get(link)
  soup = bs(r.content)
  sp = soup.find(class_="infobox vevent")

  data = {}

  try:
    for indx,i in enumerate(sp.find_all('tr')):
      if (indx == 0):                                      # Title
        data['title'] =  i.text

      elif (indx == 1):                                    # Image
        data['poster'] =  i.find('a').get('href')

      elif (i.find('li')):   
        try:                                              # Keys with Multiple Values
          data[(i.find('th').text.split(' ')[0])] = [j.text.replace('\xa0',' ') for j in i.find_all('li')]
        except:
          pass

      else:   
        try:                                             # Keys with Single Values
          data[i.find('th').text.split(' ')[0]] = i.find('td').text.replace('\xa0','')
        except:
          pass

    return data
  
  except:
    pass

  extract_data('https://en.wikipedia.org//wiki/Academy_Award_Review_of_Walt_Disney_Cartoons')

In [ ]:
r = requests.get('https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films')
soup = bs(r.content)

links = []

for i in soup.find_all(('table' ,'wikitable sortable jquery-tablesorter'))[:-3]:
  d = [m.find('a') for m in i.find_all('tr')][1:]
  
  try:
    links += [[m.text ,'https://en.wikipedia.org/'+m.get('href')] for m in d]
  except:
    pass

In [ ]:
df = pd.DataFrame(links, columns = ['title','link'])

df = df[df['title'] != '[24]']
df = df[df['title'] != '[8]']

df.head()

,title,link
0,Academy Award Review of Walt Disney Cartoons,https://en.wikipedia.org//wiki/Academy_Award_R...
1,Snow White and the Seven Dwarfs,https://en.wikipedia.org//wiki/Snow_White_and_...
2,Pinocchio,https://en.wikipedia.org//wiki/Pinocchio_(1940...
3,Fantasia,https://en.wikipedia.org//wiki/Fantasia_(1940_...
4,The Reluctant Dragon,https://en.wikipedia.org//wiki/The_Reluctant_D...


In [ ]:
js = {}

for indx,i in tqdm(enumerate(df['link'])):
  if (extract_data(i)):
    data = extract_data(i)

    key = 'd' + str(10001 + indx)

    js[key] = data

378it [01:56,  3.25it/s]


In [ ]:
corp = json.dumps(js)

fd = open('disney.json','w')
fd.write(corp)
fd.close()